In [ ]:
'''
keras: 2.3.1
tensorflow: 2.1.0
h5py: 2.10.0
'''
!pip install 'keras==2.3.1' --force-reinstall

In [ ]:
!pip install 'tensorflow==2.1.0' --force-reinstall

In [ ]:
#we also need to downgrade h5py to prevent this: https://github.com/tensorflow/tensorflow/issues/44467
!pip install 'h5py==2.10.0' --force-reinstall

#restart runtime

In [ ]:
!pip install matchzoo

In [5]:
#https://github.com/NTMC-Community/MatchZoo/blob/master/tutorials/wikiqa/init.ipynb
import keras
import pandas as pd
import numpy as np
import matchzoo as mz
import json

Using TensorFlow backend.


In [8]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task='ranking')
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision(),
    mz.metrics.AveragePrecision(),
    mz.metrics.MeanReciprocalRank(),
    mz.metrics.Precision()

]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

print("loading embedding ...")
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
print("embedding loaded as `glove_embedding`")

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0), average_precision(0.0), mean_reciprocal_rank(0.0), precision@1(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [9]:
#view the data
train_pack_raw.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0


In [10]:

def append_params_to_readme(model):
    import tabulate
    
    with open('README.rst', 'a+') as f:
        subtitle = model.params['model_class'].__name__
        line = '#' * len(subtitle)
        subtitle = subtitle + '\n' + line + '\n\n'
        f.write(subtitle)
        
        df = model.params.to_frame()[['Name', 'Value']]
        table = tabulate.tabulate(df, tablefmt='rst', headers='keys') + '\n\n'
        f.write(table)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
#source: https://github.com/NTMC-Community/MatchZoo/blob/master/tutorials/wikiqa/drmm.ipynb
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

In [13]:
preprocessor.context

{'embedding_input_dim': 17483,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f5f6feebf10>,
 'input_shapes': [(10,), (100,)],
 'vocab_size': 17483,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f5f6bb36910>}

In [14]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision(),
    mz.metrics.AveragePrecision(),
    mz.metrics.MeanReciprocalRank(),
    mz.metrics.Precision()
]

In [15]:
bin_size = 30
model = mz.models.DRMM()
model.params.update(preprocessor.context)
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()
model.backend.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5244900     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
____________________________________________________________________________________________

In [16]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]
model.load_embedding_matrix(embedding_matrix)

In [17]:
hist_callback = mz.data_generator.callbacks.Histogram(embedding_matrix, bin_size=30, hist_mode='LCH')

In [18]:
#Validation dataset for "Evaluate" callback
pred_generator = mz.DataGenerator(dev_pack_processed, mode='point', callbacks=[hist_callback])
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [19]:
train_generator = mz.DataGenerator(train_pack_processed, mode='pair', num_dup=5, num_neg=10, batch_size=20, 
                                   callbacks=[hist_callback])
print('num batches:', len(train_generator))

num batches: 255


In [20]:
%%time
history = model.fit_generator(train_generator, epochs=10, callbacks=[evaluate], workers=30, use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
255/255 [==============================] - 102s 402ms/step - loss: 2.2382
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6227526164797665 - normalized_discounted_cumulative_gain@5(0.0): 0.6732832930442673 - mean_average_precision(0.0): 0.6223807419611992 - average_precision(0.0): 0.2800332139510918 - mean_reciprocal_rank(0.0): 0.6277890896851208 - precision@1(0.0): 0.45081967213114754
Epoch 2/10
255/255 [==============================] - 101s 397ms/step - loss: 1.8288
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6247795792706264 - normalized_discounted_cumulative_gain@5(0.0): 0.6806465329923468 - mean_average_precision(0.0): 0.6320707184436694 - average_precision(0.0): 0.282270281085354 - mean_reciprocal_rank(0.0): 0.6374909926139435 - precision@1(0.0): 0.48360655737704916
Epoch 3/10
255/255 [==============================] - 102s 400ms/step - loss: 1.4946
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6007503424142793 - normalized_di

In [25]:
drmm_model = mz.load_model('./drmm_pretrained_model/2')
test_generator = mz.DataGenerator(data_pack=test_pack_processed, mode='point', callbacks=[hist_callback])
test_x, test_y = test_generator[:]

In [26]:
%%time
drmm_model.evaluate(test_x, test_y)

CPU times: user 408 ms, sys: 8.73 ms, total: 417 ms
Wall time: 407 ms


{average_precision(0.0): 0.27454071749207065,
 mean_average_precision(0.0): 0.6124554048089337,
 mean_reciprocal_rank(0.0): 0.6201458064563053,
 normalized_discounted_cumulative_gain@3(0.0): 0.6003455874772065,
 normalized_discounted_cumulative_gain@5(0.0): 0.6578222650653642,
 precision@1(0.0): 0.4430379746835443}